In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.nn import Module, LSTM, Dropout, Linear, Sigmoid, BCEWithLogitsLoss
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, auc, precision_recall_curve, classification_report

In [2]:
# Constants
DATASET_PATH = "../../data/parkinsons_dataset.csv"
TARGET_NAME = DATASET_PATH.split("/")[-1].split("_")[0]
TARGET_CLASS = "class_" + TARGET_NAME + "_w8"

LEARNING_RATE = 0.001
MAX_EPOCHS = 100
BATCH_SIZE = 32
BIDIRECTIONAL = True
DROPOUT = 0.5
LSTM_UNITS = 100
DENSE_UNITS = 1
ACTIVATION = 'sigmoid'
LOSS = BCEWithLogitsLoss()
METRICS = ['accuracy']
VALIDATION_SPLIT = 0.1
PATIENCE = 10
NUM_LAYERS = 2
TEST_SIZE = 0.1

print(f"Dataset: {DATASET_PATH}")
print(f"Target: {TARGET_NAME}")
print(f"Target class: {TARGET_CLASS}")

Dataset: ../../data/parkinsons_dataset.csv
Target: parkinsons
Target class: class_parkinsons_w8


In [3]:
data = pd.read_csv(DATASET_PATH)
data.replace('?', np.nan, inplace=True)
data = data.apply(pd.to_numeric, errors='coerce')
data.fillna(0, inplace=True)

In [25]:
target_wave_suffix = TARGET_CLASS.split('_')[-1] 
target_wave_number = int(target_wave_suffix[1:])
print(f"Target wave number: {target_wave_number}")
print(f"Target wave suffix: {target_wave_suffix}")

# Remove class variables except the target wave
class_vars_to_remove = [col for col in data.columns if col.startswith("class_") and TARGET_CLASS not in col]
features_to_remove = [col for col in data.columns if any(col.endswith(f'w{wave}') for wave in range(target_wave_number + 1, 9))]

# Combine lists to remove both class variables and features from later waves
columns_to_remove = class_vars_to_remove + features_to_remove

data_copy = data.drop(columns=columns_to_remove)


Target wave number: 8
Target wave suffix: w8


In [5]:
# Separate features and target variable
X = data_copy.drop(columns=[TARGET_CLASS])
y = data_copy[TARGET_CLASS]
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

X shape: (7097, 140)
y shape: (7097,)


In [6]:
# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [26]:
# Identify features by wave
column_names = X.columns
wave_identifiers = sorted(set(col.split('_')[-1] for col in column_names if col != 'sex' and col != 'indager_wave8' and col != 'dheas_wave4' and col != 'apoe_wave2'))
non_longitudinal_features = [col for col in column_names if not col.endswith('w1') and not col.endswith('w2') and not col.endswith('w3') and not col.endswith('w4') and not col.endswith('w5') and not col.endswith('w6') and not col.endswith('w7') and not col.endswith('w8')]

print(f"Wave identifiers: {wave_identifiers}")
print(len(non_longitudinal_features))

Wave identifiers: ['w2', 'w4', 'w6', 'w8']
4


In [8]:
# Group features by waves
features_by_wave = {wave: [] for wave in wave_identifiers}
for col in column_names:
    if col not in non_longitudinal_features:
        wave = col.split('_')[-1]
        features_by_wave[wave].append(col)
        print(wave, col)
        

w2 bmiobe_w2
w4 bmiobe_w4
w6 bmiobe_w6
w2 cfib_w2
w4 cfib_w4
w6 cfib_w6
w8 cfib_w8
w2 chestin_w2
w4 chestin_w4
w6 chestin_w6
w2 chol_w2
w4 chol_w4
w6 chol_w6
w8 chol_w8
w2 clotb_w2
w4 clotb_w4
w6 clotb_w6
w8 clotb_w8
w2 diaval_w2
w4 diaval_w4
w6 diaval_w6
w8 diaval_w8
w2 eyesurg_w2
w4 eyesurg_w4
w6 eyesurg_w6
w2 fglu_w2
w4 fglu_w4
w6 fglu_w6
w8 fglu_w8
w2 hastro_w2
w4 hastro_w4
w6 hastro_w6
w2 hasurg_w2
w4 hasurg_w4
w6 hasurg_w6
w2 hba1c_w2
w4 hba1c_w4
w6 hba1c_w6
w8 hba1c_w8
w2 hdl_w2
w4 hdl_w4
w6 hdl_w6
w8 hdl_w8
w2 hgb_w2
w4 hgb_w4
w6 hgb_w6
w8 hgb_w8
w2 hipval_w2
w4 hipval_w4
w2 hscrp_w2
w4 hscrp_w4
w6 hscrp_w6
w8 hscrp_w8
w2 htfev_w2
w4 htfev_w4
w6 htfev_w6
w2 htfvc_w2
w4 htfvc_w4
w6 htfvc_w6
w2 htpf_w2
w4 htpf_w4
w2 htval_w2
w4 htval_w4
w6 htval_w6
w4 igf1_w4
w6 igf1_w6
w8 igf1_w8
w2 ldl_w2
w4 ldl_w4
w6 ldl_w6
w8 ldl_w8
w2 mapval_w2
w4 mapval_w4
w6 mapval_w6
w8 mapval_w8
w4 mch_w4
w6 mch_w6
w8 mch_w8
w2 mmcrre_w2
w4 mmcrre_w4
w6 mmcrre_w6
w2 mmftre2_w2
w4 mmftre2_w4
w6 mmftre2_w6

In [9]:
# Prepare data for RNN
n_samples = X_scaled.shape[0]
n_timesteps = len(wave_identifiers)
n_features_per_wave = {wave: len(features) for wave, features in features_by_wave.items()}
max_features = max(n_features_per_wave.values())

print(f"Number of samples: {n_samples}")
print(f"Number of waves: {n_timesteps}")
print(f"Wave identifiers: {wave_identifiers}")
print(f"Number of features per wave: {n_features_per_wave}")

Number of samples: 7097
Number of waves: 4
Wave identifiers: ['w2', 'w4', 'w6', 'w8']
Number of features per wave: {'w2': 37, 'w4': 40, 'w6': 38, 'w8': 21}


In [27]:
# Reshape data without non-longitudinal features
X_reshaped = np.zeros((n_samples, n_timesteps, max_features))
for i, wave in enumerate(wave_identifiers):
    wave_features = features_by_wave[wave]
    indices = [column_names.get_loc(f) for f in wave_features]
    X_reshaped[:, i, :len(indices)] = X_scaled[:, indices]
    print(f"Wave {wave} features: {len(wave_features)}")
print(f"X reshaped shape: {X_reshaped.shape}")

Wave w2 features: 37
Wave w4 features: 40
Wave w6 features: 38
Wave w8 features: 21
X reshaped shape: (7097, 4, 40)


In [11]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=TEST_SIZE, random_state=42)

# PyTorch Dataset and DataLoader
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train.values, dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test.values, dtype=torch.float32))

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [12]:
print(f"X shape: {X_train.shape}")
print(f"y shape: {y_train.shape}")
print(f"Wave identifiers: {wave_identifiers}")
print(f"Number of features per wave: {n_features_per_wave}")

X shape: (6387, 4, 40)
y shape: (6387,)
Wave identifiers: ['w2', 'w4', 'w6', 'w8']
Number of features per wave: {'w2': 37, 'w4': 40, 'w6': 38, 'w8': 21}


In [13]:
class LSTMModel(Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, bidirectional=False, dropout=0.0):
        super(LSTMModel, self).__init__()
        if n_layers > 1:
            self.lstm = LSTM(input_dim, hidden_dim, n_layers, batch_first=True, bidirectional=bidirectional, dropout=dropout)
        else:
            self.lstm = LSTM(input_dim, hidden_dim, n_layers, batch_first=True, bidirectional=bidirectional)
        
        self.dropout = Dropout(dropout) if dropout > 0 else None
        direction_factor = 2 if bidirectional else 1
        self.fc = Linear(hidden_dim * direction_factor, output_dim)
        self.sigmoid = Sigmoid()

    def forward(self, x):
        h_lstm, _ = self.lstm(x)
        if self.dropout is not None:
            h_lstm = self.dropout(h_lstm[:, -1, :])  # Apply dropout only if defined
        else:
            h_lstm = h_lstm[:, -1, :]  # Take the last output from the LSTM
        out = self.fc(h_lstm)
        out = self.sigmoid(out)
        return out


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = X_train.shape[2]
hidden_dim = LSTM_UNITS
output_dim = DENSE_UNITS
n_layers = NUM_LAYERS

model = LSTMModel(input_dim, hidden_dim, output_dim, n_layers, BIDIRECTIONAL, DROPOUT).to(device)
criterion = LOSS
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

In [15]:
# Training function
def train_model(model, train_loader, criterion, optimizer, n_epochs, device):
    model.train()
    for epoch in range(n_epochs):
        running_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred.squeeze(), y_batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * X_batch.size(0)


In [16]:
# Evaluation function
def evaluate_model(model, test_loader, device):
    model.eval()
    y_true = []
    y_pred = []
    
    with torch.no_grad(): 
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_true.extend(y_batch.cpu().numpy())
            y_pred_batch = model(X_batch)
            y_pred.extend(y_pred_batch.cpu().numpy())

    y_pred = np.array(y_pred).squeeze()
    
    # Convert predictions to binary values
    y_pred_binary = (y_pred > 0.5).astype(int)
    
    # Calculate metrics using binary predictions
    precision = precision_score(y_true, y_pred_binary)
    recall = recall_score(y_true, y_pred_binary)
    f1 = f1_score(y_true, y_pred_binary)
    
    # ROC-AUC and Precision-Recall AUC use the continuous y_pred
    roc_auc = roc_auc_score(y_true, y_pred)
    precision_curve, recall_curve, _ = precision_recall_curve(y_true, y_pred)
    auprc = auc(recall_curve, precision_curve)
    
    report = classification_report(y_true, y_pred_binary)
    conf_matrix = confusion_matrix(y_true, y_pred_binary)

    print(f'Test Precision: {precision:.4f}')
    print(f'Test Recall: {recall:.4f}')
    print(f'Test F1 Score: {f1:.4f}')
    print(f'Test ROC-AUC Score: {roc_auc:.4f}')
    print(f'Test AUPRC Score: {auprc:.4f}')
    print(f'Confusion Matrix:\n{conf_matrix}')
    print(f'Classification Report:\n{report}')
    return y_true, y_pred

In [17]:
# # Training the model
# train_model(model, train_loader, criterion, optimizer, 100, device)
# evaluate_model(model, test_loader, device)

In [18]:
from itertools import product

# Define hyperparameter grid
learning_rates = [0.001, 0.01]
dropout_rates = [0.2, 0.5]
n_layers_options = [1, 2]
lstm_units_options = [50, 100]

# All combinations of hyperparameters
hyperparameter_grid = list(product(learning_rates, dropout_rates, n_layers_options, lstm_units_options))


In [19]:
from sklearn.model_selection import KFold

def train_validate_model(model, train_loader, val_loader, criterion, optimizer, n_epochs, device):
    """Train the model and evaluate on validation set."""
    model.train()
    for epoch in range(n_epochs):
        running_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred.squeeze(), y_batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * X_batch.size(0)
    
    # Evaluate on validation set
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_true.extend(y_batch.cpu().numpy())
            y_pred_batch = model(X_batch)
            y_pred.extend(y_pred_batch.cpu().numpy())
    
    y_pred = np.array(y_pred).squeeze()
    y_pred_binary = (y_pred > 0.5).astype(int)
    
    precision = precision_score(y_true, y_pred_binary)
    recall = recall_score(y_true, y_pred_binary)
    f1 = f1_score(y_true, y_pred_binary)
    
    conf_matrix = confusion_matrix(y_true, y_pred_binary)
    
    return f1, conf_matrix, recall, precision

def grid_search(train_data, val_data, hyperparameter_grid, device, n_epochs=10):
    best_hyperparameters = None
    best_f1 = float('-inf')
    best_conf_matrix = None
    best_preicison = None
    best_recall = None

    for lr, dropout, n_layers, lstm_units in hyperparameter_grid:
        print(f"Training with learning rate: {lr}, dropout: {dropout}, layers: {n_layers}, units: {lstm_units}")
        model = LSTMModel(input_dim, lstm_units, output_dim, n_layers, BIDIRECTIONAL, dropout).to(device)
        optimizer = Adam(model.parameters(), lr=lr)
        criterion = BCEWithLogitsLoss()
        
        # Prepare data loaders
        train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)
        
        # Train and evaluate
        f1, conf_matrix, recall, precision = train_validate_model(model, train_loader, val_loader, criterion, optimizer, n_epochs, device)
        
        # Save the best hyperparameters based on AUC score
        if f1 > best_f1:
            best_f1 = f1
            best_conf_matrix = conf_matrix
            best_hyperparameters = (lr, dropout, n_layers, lstm_units)
            best_preicison = precision
            best_recall = recall
            print(f"Found new best hyperparameters: {best_hyperparameters}")

    return best_hyperparameters, best_f1, best_conf_matrix, best_preicison, best_recall

# K-Fold Cross Validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
all_fold_results = []

best_results_per_fold = []

for fold, (train_index, test_index) in enumerate(kf.split(X_reshaped)):
    print(f'Fold {fold + 1}')
    
    # Split data into train and test sets for this fold
    X_train_fold, X_test_fold = X_reshaped[train_index], X_reshaped[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    
    # Further split train into learning and validation sets
    X_train_sub, X_val_sub, y_train_sub, y_val_sub = train_test_split(X_train_fold, y_train_fold, test_size=VALIDATION_SPLIT, random_state=42)
    
    # Convert to TensorDatasets
    train_data_sub = TensorDataset(torch.tensor(X_train_sub, dtype=torch.float32), torch.tensor(y_train_sub.values, dtype=torch.float32))
    val_data_sub = TensorDataset(torch.tensor(X_val_sub, dtype=torch.float32), torch.tensor(y_val_sub.values, dtype=torch.float32))
    
    # Perform grid search on the learning and validation set
    best_hyperparameters, best_f1, best_conf_matrix, best_preicison, best_recall = grid_search(train_data_sub, val_data_sub, hyperparameter_grid, device, n_epochs=MAX_EPOCHS)
    
    print(f"Best Hyperparameters for Fold {fold + 1}: {best_hyperparameters}")
    
    best_results_per_fold.append([best_hyperparameters, best_f1, best_conf_matrix, best_preicison, best_recall])
    
    # Now train on full training set (X_train_fold) with the best hyperparameters
    lr, dropout, n_layers, lstm_units = best_hyperparameters
    print(f"Training on full training set with best hyperparameters: {best_hyperparameters}")
    model = LSTMModel(input_dim, lstm_units, output_dim, n_layers, BIDIRECTIONAL, dropout).to(device)
    optimizer = Adam(model.parameters(), lr=lr)
    
    # Prepare final train and test loaders
    train_data_final = TensorDataset(torch.tensor(X_train_fold, dtype=torch.float32), torch.tensor(y_train_fold.values, dtype=torch.float32))
    test_data_final = TensorDataset(torch.tensor(X_test_fold, dtype=torch.float32), torch.tensor(y_test_fold.values, dtype=torch.float32))
    
    train_loader_final = DataLoader(train_data_final, batch_size=BATCH_SIZE, shuffle=True)
    test_loader_final = DataLoader(test_data_final, batch_size=BATCH_SIZE, shuffle=False)
    
    # Train on the full training set
    train_model(model, train_loader_final, criterion, optimizer, MAX_EPOCHS, device)
    
    # Evaluate on the test set
    results = evaluate_model(model, test_loader_final, device)


Fold 1
Training with learning rate: 0.001, dropout: 0.2, layers: 1, units: 50


c:\Users\Grego\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Found new best hyperparameters: (0.001, 0.2, 1, 50)
Training with learning rate: 0.001, dropout: 0.2, layers: 1, units: 100


c:\Users\Grego\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training with learning rate: 0.001, dropout: 0.2, layers: 2, units: 50


...

c:\Users\Grego\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training with learning rate: 0.01, dropout: 0.5, layers: 2, units: 100


c:\Users\Grego\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Best Hyperparameters for Fold 10: (0.001, 0.2, 1, 50)
Training on full training set with best hyperparameters: (0.001, 0.2, 1, 50)
Test Precision: 0.0000
Test Recall: 0.0000
Test F1 Score: 0.0000
Test ROC-AUC Score: 0.5277
Test AUPRC Score: 0.0084
Confusion Matrix:
[[705   0]
 [  4   0]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00       705
         1.0       0.00      0.00      0.00         4

    accuracy                           0.99       709
   macro avg       0.50      0.50      0.50       709
weighted avg       0.99      0.99      0.99       709



c:\Users\Grego\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Grego\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Grego\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [24]:
list_of_dicts = []
def format_confusion_matrix(conf_matrix):
    return f"[[{conf_matrix[0][0]}, {conf_matrix[0][1]}]\n [{conf_matrix[1][0]}, {conf_matrix[1][1]}]]"

for fold, i in enumerate(best_results_per_fold):
    cleaned_hyperparameters = i[0]
    cleaned_conf_matrix = i[2]
    # i[1] round to 4 decimal places
    converted_f1 = round(i[1], 4)
    convert_precision = round(i[3], 4)
    convert_recall = round(i[4], 4)
    print(cleaned_hyperparameters)
    #print(format_confusion_matrix(cleaned_conf_matrix))
    list_of_dicts.append({'Hyperparameters': cleaned_conf_matrix})

# Print the DataFrame without the index
print(pd.DataFrame(list_of_dicts).to_string(index=False))

print(f"Dataset name is: {DATASET_PATH}")

(0.001, 0.2, 1, 50)
(0.001, 0.2, 1, 50)
(0.001, 0.2, 1, 50)
(0.001, 0.2, 1, 50)
(0.001, 0.2, 1, 50)
(0.001, 0.2, 1, 50)
(0.001, 0.2, 1, 50)
(0.001, 0.2, 1, 50)
(0.001, 0.2, 1, 50)
(0.001, 0.2, 1, 50)
   Hyperparameters
[[632, 0], [7, 0]]
[[631, 0], [8, 0]]
[[636, 0], [3, 0]]
[[636, 0], [3, 0]]
[[635, 0], [4, 0]]
[[631, 0], [8, 0]]
[[631, 0], [8, 0]]
[[637, 0], [2, 0]]
[[634, 0], [5, 0]]
[[632, 0], [7, 0]]
Dataset name is: ../../data/parkinsons_dataset.csv
